In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import os

import flax
from flax import jax_utils
import jax
import jax.numpy as jnp
import numpy as np
import optax
import torch
import tensorflow as tf
import tqdm

import init2winit
import fastmri

import i2w

from fastmri.models import unet as t_unet
from fastmri.pl_modules import data_module
from fastmri.pl_modules import unet_module
from fastmri.data.transforms import UnetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

from init2winit.model_lib import unet as f_unet
from init2winit.dataset_lib import fastmri_dataset
from init2winit.dataset_lib import data_utils
from init2winit.optimizer_lib import optimizers

jax.devices()

[GpuDevice(id=0, process_index=0),
 GpuDevice(id=1, process_index=0),
 GpuDevice(id=2, process_index=0),
 GpuDevice(id=3, process_index=0),
 GpuDevice(id=4, process_index=0),
 GpuDevice(id=5, process_index=0),
 GpuDevice(id=6, process_index=0),
 GpuDevice(id=7, process_index=0)]

# Data pipeline

In [8]:
data_path = pathlib.Path('../../')
challenge = 'singlecoil'
center_fractions = [0.08]
accelerations = [4]
mask_type = None  # Should be `random`, but tying out without
test_split = 'test'

## `fastmri`

In [9]:
# mask = create_mask_for_mask_type(mask_type, center_fractions, accelerations)
mask = None

train_transform = UnetDataTransform(challenge, mask_func=mask, use_seed=False)
val_transform = UnetDataTransform(challenge, mask_func=mask)
test_transform = UnetDataTransform(challenge)

dm = data_module.FastMriDataModule(
    data_path=data_path,
    challenge=challenge,
    train_transform=train_transform,
    val_transform=val_transform,
    test_transform=test_transform,
    test_split=test_split,
    batch_size=1,
    num_workers=4,
)

dl = iter(dm._create_data_loader(train_transform, data_partition='train'))

## `init2winit`

In [11]:
for i, t_batch in enumerate(tqdm.tqdm(dl)):
    inputs = i2w.get_slice(os.path.join('../../singlecoil_train', t_batch.fname[0]), t_batch.slice_num[0])
    f_batch = fastmri_dataset._process_example(*inputs, tf.cast(jax.random.PRNGKey(0), tf.int64))
    
    input_delta = np.mean(np.abs(f_batch['inputs'].numpy() - t_batch.image.numpy().squeeze()))
    output_delta = np.mean(np.abs(f_batch['targets'].numpy() - t_batch.target.numpy().squeeze()))
    vol_delta = f_batch['volume_max'] - t_batch.max_value.numpy().squeeze()
    
    if input_delta > 1e-5 or output_delta > 1e-5:
        print(i, input_delta, output_delta, vol_delta)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 34741/34742 [12:13<00:00, 47.34it/s]
